In [2]:
%load_ext autoreload
%autoreload 2

import rdkit
rdkit.Chem.Draw.IPythonConsole.ipython_maxProperties = -1

import dgym as dg

# load all data
print('load data')
path = '../../dgym-data'

deck = dg.MoleculeCollection.from_sdf(
    f'{path}/DSi-Poised_Library_annotated.sdf',
    reactant_names=['reagsmi1', 'reagsmi2', 'reagsmi3']
)

load data


In [26]:
%load_ext autoreload
%autoreload 2

import rdkit
rdkit.Chem.Draw.IPythonConsole.ipython_maxProperties = -1

import dgym as dg

# load all data
print('load data')
path = '../../dgym-data'

deck = dg.MoleculeCollection.from_sdf(
    f'{path}/Mpro_combined_aminopyridine.sdf',
    reactant_names=['reagsmi1', 'reagsmi2', 'reagsmi3']
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
load data


In [27]:
from meeko import MoleculePreparation
from meeko import PDBQTWriterLegacy

def get_pdbqt(mol):

    # add hydrogens (without regard to pH)
    protonated_mol = rdkit.Chem.AddHs(mol)

    # generate 3D coordinates for the ligand. 
    rdkit.Chem.AllChem.EmbedMolecule(protonated_mol)

    # intialize preparation
    preparator = MoleculePreparation(
        # keep_chorded_rings=True,
        # keep_equivalent_rings=True,
        rigid_macrocycles=True
    )
    mol_setups = preparator.prepare(protonated_mol)
    
    for setup in mol_setups:
        pdbqt_string, is_ok, error_msg = PDBQTWriterLegacy.write_string(setup)
        if is_ok:
            return pdbqt_string

Create ligand PDBQTs.

In [28]:
import os
from tqdm.notebook import tqdm

out_dir = '../../dgym-data/out/ligands_temp/'

paths = []
for mol in tqdm(deck[:]):
    path = os.path.join(out_dir, f'{mol.name}.pdbqt')
    paths.append(path)
    pdbqt = get_pdbqt(mol.mol)
    
    with open(path, 'w') as file:
        file.write(pdbqt)

ligands_txt = ' '.join(paths)
path = os.path.join(out_dir, 'ligands.txt')
with open(path, 'w') as file:
    file.write(ligands_txt)

  0%|          | 0/158 [00:00<?, ?it/s]

Create ligand txt summary file.

In [29]:
# import subprocess
# import tempfile

# with tempfile.TemporaryDirectory() as tempdir:
#     print(f"Temporary directory created at {tempdir}")

In [192]:
%%time
import subprocess

command = 'unidock --receptor ../../dgym-data/Mpro_prepped.pdbqt --ligand_index ../../dgym-data/out/ligands_temp/ligands.txt --center_x 9.812 --center_y -0.257 --center_z 20.8485 --size_x 14.328 --size_y 8.85 --size_z 12.539 --dir ../../dgym-data/out/docking_results --exhaustiveness 128 --max_step 20 --num_modes 9 --scoring vinardo --refine_step 3 --seed 5'
resp = subprocess.run(
    command,
    shell=True, 
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE, 
    encoding='utf-8'
)

In [193]:
import numpy as np

def boltzmann_sum(energies, temperature=298.15):
    kT = 0.0019872041 * temperature  # Boltzmann constant in kcal/(mol·K) multiplied by temperature in K
    energies = np.array(energies)  # Energies should be in kcal/mol

    # Use logsumexp for numerical stability
    return -kT * np.log(np.sum(np.exp(-energies / kT)))

In [194]:
import pandas as pd
import re
import glob
from itertools import islice
from operator import itemgetter

affinities = []
paths = glob.glob('../../dgym-data/out/docking_results/*.pdbqt')

for idx, path in enumerate(paths):
    with open(path, 'r') as file:

        # extract SMILES from the file
        smiles_str = list(islice(file, 7))[-1]
        smiles = smiles_str.split(' ')[-1].split('\n')[0]
        file.seek(0)

        # extract affinities
        affinity_strs = [line for line in file
                         if line.startswith('REMARK VINA RESULT')]
        process_affinity = lambda s: re.search(r'-?\d+\.\d+', s).group()
        affinity = [float(process_affinity(a)) for a in affinity_strs]

        # compute boltzmann sum
        affinity = boltzmann_sum(affinity)

        # append to affinities
        affinities.append(
            {'smiles': smiles, 'aff_pred': affinity}
        )

affinities = pd.DataFrame(affinities)

In [199]:
affinities

,smiles,aff_pred
0,COc1cc(Cl)cc(CC(=O)Nc2cnccc2C)c1,-5.472481
1,N#Cc1cncc(NC(=O)Cc2cccnc2)c1,-6.219125
2,Cn1ncc2cncc(NC(=O)Cc3cccc(Cl)c3)c21,-6.528216
3,CC(=O)N1CC[NH+](CC(=O)Nc2cnccc2C)CC1,-5.963856
4,N#Cc1cncc(CC(=O)Nc2cccnc2)c1,-6.114066
...,...,...
140,Cc1ccncc1NC(=O)Cc1cc(Cl)cc(N[C@H]2C[C@H]2C(F)(...,-4.473278
141,Cc1ccncc1NC(=O)C[NH+]1CCCCC1,-5.333700
142,CC(=O)NCc1cc(Cl)cc(CC(=O)Nc2cnccc2C)c1,-4.940242
143,Cc1ccncc1NC(=O)Cc1cccc(C(F)(F)F)c1,-5.968522


In [240]:
from dgym.envs.oracle import DockingOracle

config = {
    'center_x': 9.812,
    'center_y': -0.257,
    'center_z': 20.8485,
    'size_x': 14.328,
    'size_y': 8.85,
    'size_z': 12.539,
    'exhaustiveness': 128,
    'max_step': 20,
    'num_modes': 9,
    'scoring': 'vinardo',
    'refine_step': 3,
    'seed': 5
}

docking_oracle = DockingOracle(
    'Mpro',
    receptor_path='../../dgym-data/Mpro_prepped.pdbqt',
    config=config
)

In [245]:
docking_oracle.predict(deck[:96])

[-6.363591366337899,
 -6.803148314309183,
 -5.476738357087498,
 -5.906628311679397,
 -5.55992511119615]